# fastai vs. native torch predictions

fastai is a high level API built on top of torch providing features to simplify training models. Everything written in fastai can be converted to native torch. This could be useful if a decision is made to reducde dependencies in production *(e.g. prediction part)* .

The following sections explain the parallels between fastai and torch, focusing on the **prediction part**. The versions of the fastai/torch are and included in `requirements_fastai.txt`.

In [24]:
%%capture
from fastai.vision.all import *

import glob
import pickle

import torch
import torchvision.transforms as T
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader

from dataset_custom import ImageDataset ## custom dataset class

In [25]:
## using num_workers=0 for local testing (but can be changed to more if desired)
PRED_NUM_WORKERS = 0

## specify model and test data
model_path = 'models'
model_name = 'model_modified.pkl'
test_data_path = 'images/imgs_zz/' ## small sample 

## fastai (prediction)

We'll start by using a model that has already been trained and exported as `.pkl` using fastai (2.5.5).

### Loading models in fastai

To load models in fastai, `load_learner()` is used. The loaded model includes more details than models exprorted in native torch. For instance, the dataloader object includes the transformations specified at the time of model training and other details such as the list of tags.

In [26]:
## load model 
learn_multi = load_learner(f'{model_path}/{model_name}')

### Batch prediction in fastai

To predict on a batch, the following steps should be followed:

- get paths of images.
- load images in batches and the corresponding transformation using `test_dl`.
- predict on batches and return all predictions.


Note that the default batch size is 64, but we pass another value to `bs`. his value can be retrieved using `tst_dl.bs`.

In [27]:
## predict on batch
tst_files = get_image_files(test_data_path)
tst_dl = learn_multi.dls.test_dl(tst_files, bs=4, num_workers=PRED_NUM_WORKERS)
preds_fastai, _ = learn_multi.get_preds(dl=tst_dl)

### Data in loaded model and dataloader (details)

#### Tags names

To retrieve tag names with the order returned by the predictions functions, use `learn_multi.dls.vocab` as follows.

In [28]:
%%capture
learn_multi.dls.vocab

#### Transformations

To retrieve the image transformations applied on the images, check:
- `tst_dl.after_item` (applied on each item)
- `tst_dl.after_batch` (applied on each batch)


**\*.after_item()**

Looking at the item transforms, we can see that the image gets resized to 460x460, with method squish for train data. However, aaccording to the [docs](https://docs.fast.ai/vision.augment.html#Resize), valid or test data always get center cropped.

In [29]:
tst_dl.after_item

Pipeline: Resize -- {'size': (460, 460), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0} -> ToTensor

**\*.after_batch()**

The batch transformtions here are mainly `aug_transforms()`, which is applied on training data and not relevant to test data.

In [30]:
tst_dl.after_batch

Pipeline: IntToFloatTensor -- {'div': 255.0, 'div_mask': 1} -> Warp -- {'magnitude': 0.2, 'p': 1.0, 'draw_x': None, 'draw_y': None, 'size': None, 'mode': 'bilinear', 'pad_mode': 'reflection', 'batch': False, 'align_corners': True, 'mode_mask': 'nearest'} -> Brightness -- {'max_lighting': 0.2, 'p': 1.0, 'draw': None, 'batch': False}

So what we care about here is the resize value which gets applied to train/valid/test data --> `tst_dl.after_item.size` or `learn_multi.dls.after_item.size`.

In [31]:
tst_dl.after_item.size

(460, 460)


According to these transformations, what we expect from the dataloader includes the following steps:

- resizing image *(to `learn_multi.dls.after_item.size`)*.
- transforming to tensor *(expected to see normalized values in the range [0,1])*.


### Zoom in on one example

Looking closer at one loaded test image `tst_dl.items[3]`, we can see the following:

In [32]:
## image path 
tst_dl.items[3]

Path('images/imgs_zz/344922.jpeg')

In [33]:
## loaded image type
tst_dl.one_batch()[0][3].type()

'torch.FloatTensor'

In [34]:
## image size before resizing
tst_dl.dataset[3]

(PILImage mode=RGB size=800x600,)

The size is similar to what's expected `learn_multi.dls.after_item.size`.

In [35]:
## image size after resizing 
test_image_sample_fastai = np.array(tst_dl.one_batch()[0][3])
test_image_sample_fastai.shape

(3, 460, 460)

The range of values is between [0,1] *(checking one channel)*.

In [36]:
## loaded image --> tensor values range
test_image_sample_fastai.min(), test_image_sample_fastai.max()

(0.0, 1.0)

The predictions values combined with tag names.

In [37]:
## predictions
preds_fastai[3]

tensor([9.2788e-04, 1.8691e-03, 2.6241e-04, 1.8735e-03, 7.3655e-01, 1.1181e-03,
        1.5422e-02, 9.6627e-05, 9.1196e-03, 6.3069e-05, 2.5095e-03, 9.8131e-05])

## torch (prediction)

Now let's see the equivalent in native torch! 

### Saving fastai models for torch (jit)
First we'll save the model in a format that can be read by native torch with no dependancy on fastai as follows.

In [38]:
## image size 
fastai_model_img_size = tst_dl.after_item.size[0]

## save for native torch import later
dummy_inp = torch.randn([1,3,fastai_model_img_size,fastai_model_img_size]) ## dummy 
torch.jit.save(torch.jit.trace(learn_multi.model, dummy_inp), 'models/model_modified_jit.pt')

## save vocab with model 
vocab = np.save('models/vocab.npy', learn_multi.dls.vocab)

### Loading models in torch
In practice, if we are just writing the prediction part, we'll start from this point where a model already exists 
Here we'll try to write the equivalent prediction code without using any functions from fastai. Since the fastai model was exported using `jit`, we'll use `torch.jit.load()`. Then we'll use `eval()` which is an important step before prediction.

In [39]:
# %%capture
## load model
model_loaded = torch.jit.load('models/model_modified_jit.pt')

## eval() is required after load() during inference/prediction
_ = model_loaded.eval()

Since the model is not a fastai model anymore, we won't have access to the resizing value, transformations or vocab. We should have these values saved when the model gets exported.

In [40]:
## specify resize value (or retrieve from a file if saved)
IMG_RESIZE_VALUE = 460

In [41]:
## specifiy test images
test_files = glob.glob(f'{test_data_path}*')

### Transformation and prediction in torch (one example)

Here we'll take the same sample image `test_files[3]` which corresponds to `tst_dl.items[3]`. We'll apply the transformations and predict using torch as follows.

In [42]:
with torch.no_grad(): ## important to use during prediction
    pil_image = Image.open(test_files[3]) # open image as PIL
    resize = T.Resize([IMG_RESIZE_VALUE, IMG_RESIZE_VALUE]) # specify resize value
    res_pil_image = resize(pil_image) # use torch native resize 
    timg = T.ToTensor()(res_pil_image) # convert to tesnor
    pred_sample_torch = model_loaded(timg.unsqueeze(0)).sigmoid() # use model to get predictions (sigmoid for multi-label multi-class)

Looking at the results, we can see the same predictions as fastai predictions.

In [43]:
pred_sample_torch[0]

tensor([9.2788e-04, 1.8691e-03, 2.6241e-04, 1.8735e-03, 7.3655e-01, 1.1181e-03,
        1.5422e-02, 9.6627e-05, 9.1196e-03, 6.3069e-05, 2.5095e-03, 9.8132e-05])

### Batch prediction in torch 

To predict on a batch, we can use a [custom dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files) to include all transformations as shown below. If we inspect the`ImageDataset` class under `dataset_custom.py`, we can see the details. The main point is adding the transformations to `__getitem__`.

In [44]:
BATCH_SIZE = 4
dataset = ImageDataset(test_files, img_size=IMG_RESIZE_VALUE)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=PRED_NUM_WORKERS) #iterator

In case of datasets including images > `batch_size`, we need to loop over the batches. 

In [45]:
final_output = []

with torch.no_grad():
    for batch in loader:
        preds = model_loaded(batch)
        preds_batch = preds.sigmoid()  
        final_output.append(preds_batch)
preds_torch = torch.cat(final_output, dim=0) 

## fastai versus torch predictions

Comparing the probablities from fastai versus torch, we can see that we got the same values.
*Note that the values should be the same, but might have slight differences after a certain N decimal points.*

In [46]:
np.array_equal(np.around(preds_torch.numpy(), 3), np.around(preds_fastai.numpy(), 3))

True